# Concurrency Basics

**Chapter 10 - Learning Python, 5th Edition**

Python provides multiple concurrency models: **threading** for I/O-bound parallelism,
**multiprocessing** for CPU-bound parallelism, and **asyncio** for cooperative
multitasking. Understanding the Global Interpreter Lock (GIL) and when each model
applies is essential for writing performant Python.

## Threading Basics

The `threading` module provides a high-level interface for creating and managing
threads. Threads share the same memory space, making data sharing easy but
requiring synchronization. Daemon threads are background threads that are killed
automatically when all non-daemon threads have exited.

In [ ]:
import threading
import time


def worker(name: str, duration: float) -> None:
    """Simulate a task that takes some time."""
    thread = threading.current_thread()
    print(f"[{name}] Starting on thread '{thread.name}' (daemon={thread.daemon})")
    time.sleep(duration)
    print(f"[{name}] Finished after {duration}s")


# Create and start threads
threads: list[threading.Thread] = []
for i, duration in enumerate([0.5, 0.3, 0.4], start=1):
    t = threading.Thread(target=worker, args=(f"Task-{i}", duration), name=f"Worker-{i}")
    threads.append(t)
    t.start()

# Wait for all threads to complete
for t in threads:
    t.join()

print(f"\nAll threads completed. Active thread count: {threading.active_count()}")


# Daemon thread example: runs in background, killed when main thread exits
def background_monitor(interval: float) -> None:
    """A daemon thread that monitors something periodically."""
    while True:
        print(f"  [Monitor] Active threads: {threading.active_count()}")
        time.sleep(interval)


monitor = threading.Thread(target=background_monitor, args=(0.2,), daemon=True)
monitor.start()

print(f"\nMonitor thread is daemon: {monitor.daemon}")
print(f"Monitor thread is alive: {monitor.is_alive()}")
time.sleep(0.5)  # Let the monitor run briefly
print("Main thread done -- daemon will be stopped automatically")

## The Global Interpreter Lock (GIL)

CPython has a Global Interpreter Lock that allows only one thread to execute
Python bytecode at a time. This means:

- **I/O-bound tasks** benefit from threading because threads release the GIL
  while waiting for I/O (network, disk, sleep).
- **CPU-bound tasks** do NOT benefit from threading. Multiple threads will
  effectively run one at a time, sometimes even slower due to context switching.
- For CPU-bound parallelism, use `multiprocessing` or `ProcessPoolExecutor`.

The demonstration below shows threading helping I/O-bound work but not CPU-bound work.

In [ ]:
import threading
import time


def io_bound_task(task_id: int) -> str:
    """Simulate I/O-bound work (network call, file read, etc.)."""
    time.sleep(0.5)  # Simulates waiting for I/O
    return f"IO-{task_id} done"


def cpu_bound_task(n: int) -> int:
    """CPU-bound work: sum of squares."""
    return sum(i * i for i in range(n))


# --- I/O-bound: sequential vs threaded ---
NUM_TASKS = 6

start = time.perf_counter()
for i in range(NUM_TASKS):
    io_bound_task(i)
sequential_io = time.perf_counter() - start

start = time.perf_counter()
threads = [threading.Thread(target=io_bound_task, args=(i,)) for i in range(NUM_TASKS)]
for t in threads:
    t.start()
for t in threads:
    t.join()
threaded_io = time.perf_counter() - start

print("=== I/O-bound (threading HELPS) ===")
print(f"Sequential: {sequential_io:.2f}s")
print(f"Threaded:   {threaded_io:.2f}s")
print(f"Speedup:    {sequential_io / threaded_io:.1f}x")


# --- CPU-bound: sequential vs threaded ---
N = 2_000_000

start = time.perf_counter()
for _ in range(NUM_TASKS):
    cpu_bound_task(N)
sequential_cpu = time.perf_counter() - start

start = time.perf_counter()
threads = [threading.Thread(target=cpu_bound_task, args=(N,)) for _ in range(NUM_TASKS)]
for t in threads:
    t.start()
for t in threads:
    t.join()
threaded_cpu = time.perf_counter() - start

print("\n=== CPU-bound (threading does NOT help due to GIL) ===")
print(f"Sequential: {sequential_cpu:.2f}s")
print(f"Threaded:   {threaded_cpu:.2f}s")
print(f"Speedup:    {sequential_cpu / threaded_cpu:.1f}x  (expect ~1x or worse)")

## Thread Synchronization with Locks

When threads share mutable state, race conditions can occur. A `Lock` ensures
only one thread can access the critical section at a time. Always use locks
as context managers (`with lock:`) to guarantee release even on exceptions.

In [ ]:
import threading


class UnsafeCounter:
    """Counter WITHOUT synchronization -- prone to race conditions."""

    def __init__(self) -> None:
        self.value: int = 0

    def increment(self) -> None:
        # Read-modify-write is NOT atomic: another thread can interleave
        current = self.value
        self.value = current + 1


class SafeCounter:
    """Counter WITH lock-based synchronization."""

    def __init__(self) -> None:
        self.value: int = 0
        self._lock = threading.Lock()

    def increment(self) -> None:
        with self._lock:  # Only one thread enters this block at a time
            current = self.value
            self.value = current + 1


def run_increments(counter: UnsafeCounter | SafeCounter, num_threads: int, per_thread: int) -> int:
    """Spawn threads to increment a shared counter."""
    def target() -> None:
        for _ in range(per_thread):
            counter.increment()

    threads = [threading.Thread(target=target) for _ in range(num_threads)]
    for t in threads:
        t.start()
    for t in threads:
        t.join()
    return counter.value


NUM_THREADS = 10
PER_THREAD = 10_000
EXPECTED = NUM_THREADS * PER_THREAD

# Unsafe counter may lose increments
unsafe = UnsafeCounter()
unsafe_result = run_increments(unsafe, NUM_THREADS, PER_THREAD)
print(f"Unsafe counter: {unsafe_result:,} (expected {EXPECTED:,}, lost {EXPECTED - unsafe_result:,})")

# Safe counter is always correct
safe = SafeCounter()
safe_result = run_increments(safe, NUM_THREADS, PER_THREAD)
print(f"Safe counter:   {safe_result:,} (expected {EXPECTED:,}, lost {EXPECTED - safe_result:,})")

## concurrent.futures: High-Level Concurrency

The `concurrent.futures` module provides a clean, unified API for both threading
and multiprocessing through `ThreadPoolExecutor` and `ProcessPoolExecutor`.
The `Future` object represents a pending result, and `as_completed` yields
futures as they finish (not in submission order).

In [ ]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor, as_completed, Future
import time


def fetch_url(url: str) -> dict[str, str | float]:
    """Simulate fetching a URL with variable latency."""
    latency = len(url) % 5 * 0.1 + 0.1  # Deterministic fake latency
    time.sleep(latency)
    return {"url": url, "status": "200 OK", "latency": round(latency, 2)}


urls: list[str] = [
    "https://api.example.com/users",
    "https://api.example.com/products",
    "https://api.example.com/orders",
    "https://api.example.com/inventory",
    "https://api.example.com/analytics",
]

# --- submit() + as_completed: process results as they arrive ---
print("=== ThreadPoolExecutor with as_completed ===")
start = time.perf_counter()

with ThreadPoolExecutor(max_workers=3) as executor:
    # Submit all tasks, mapping futures back to their input
    future_to_url: dict[Future[dict[str, str | float]], str] = {
        executor.submit(fetch_url, url): url for url in urls
    }

    for future in as_completed(future_to_url):
        url = future_to_url[future]
        try:
            result = future.result()  # Raises if the callable raised
            print(f"  {result['url']} -> {result['status']} ({result['latency']}s)")
        except Exception as exc:
            print(f"  {url} generated an exception: {exc}")

print(f"  Total time: {time.perf_counter() - start:.2f}s\n")


# --- executor.map(): simpler API, results in submission order ---
print("=== ThreadPoolExecutor with map (ordered results) ===")
start = time.perf_counter()

with ThreadPoolExecutor(max_workers=3) as executor:
    results = executor.map(fetch_url, urls)
    for result in results:  # Results arrive in submission order
        print(f"  {result['url']} -> {result['status']}")

print(f"  Total time: {time.perf_counter() - start:.2f}s")

## ProcessPoolExecutor for CPU-Bound Work

`ProcessPoolExecutor` spawns separate processes, each with its own Python
interpreter and GIL. This achieves true parallelism for CPU-bound tasks.
The API is identical to `ThreadPoolExecutor`, making it easy to swap.

In [ ]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import time
import math


def is_prime(n: int) -> bool:
    """CPU-intensive primality test."""
    if n < 2:
        return False
    if n < 4:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    for i in range(5, int(math.isqrt(n)) + 1, 6):
        if n % i == 0 or n % (i + 2) == 0:
            return False
    return True


# Large numbers to test primality
candidates: list[int] = [
    15485863, 15485867, 32452843, 32452847,
    49979687, 49979693, 67867967, 67867979,
]

# Sequential
start = time.perf_counter()
sequential_results = [is_prime(n) for n in candidates]
seq_time = time.perf_counter() - start

# Threaded (limited by GIL for CPU work)
start = time.perf_counter()
with ThreadPoolExecutor(max_workers=4) as executor:
    thread_results = list(executor.map(is_prime, candidates))
thread_time = time.perf_counter() - start

# Multiprocess (true parallelism, bypasses GIL)
start = time.perf_counter()
with ProcessPoolExecutor(max_workers=4) as executor:
    process_results = list(executor.map(is_prime, candidates))
process_time = time.perf_counter() - start

print("=== CPU-bound: Primality Testing ===")
print(f"Sequential:        {seq_time:.4f}s")
print(f"ThreadPool (4w):   {thread_time:.4f}s  (speedup: {seq_time/thread_time:.2f}x)")
print(f"ProcessPool (4w):  {process_time:.4f}s  (speedup: {seq_time/process_time:.2f}x)")

print("\nResults:")
for n, prime in zip(candidates, process_results):
    print(f"  {n:>10,} -> {'prime' if prime else 'composite'}")

## Async/Await with asyncio

The `asyncio` module provides cooperative multitasking using coroutines.
Unlike threads, coroutines run on a single thread and yield control
explicitly with `await`. This avoids race conditions and is ideal
for high-concurrency I/O (thousands of simultaneous connections).

Key concepts:
- **coroutine**: Defined with `async def`, must be awaited
- **await**: Suspends the coroutine until the awaited operation completes
- **gather**: Runs multiple coroutines concurrently
- **event loop**: Schedules and runs coroutines (managed automatically)

In [ ]:
import asyncio
import time


async def fetch_data(source: str, delay: float) -> dict[str, str | float]:
    """Simulate an async I/O operation (e.g., HTTP request, DB query)."""
    print(f"  [{source}] Starting fetch...")
    await asyncio.sleep(delay)  # Non-blocking sleep; yields control to event loop
    print(f"  [{source}] Completed after {delay}s")
    return {"source": source, "data": f"result from {source}", "latency": delay}


async def main_sequential() -> list[dict[str, str | float]]:
    """Run coroutines one at a time (no concurrency)."""
    results = []
    for source, delay in [("DB", 0.5), ("API", 0.3), ("Cache", 0.1)]:
        result = await fetch_data(source, delay)  # Waits for each one
        results.append(result)
    return results


async def main_concurrent() -> list[dict[str, str | float]]:
    """Run coroutines concurrently with gather."""
    results = await asyncio.gather(
        fetch_data("DB", 0.5),
        fetch_data("API", 0.3),
        fetch_data("Cache", 0.1),
    )
    return list(results)


# Sequential execution
print("=== Sequential (await one at a time) ===")
start = time.perf_counter()
seq_results = await main_sequential()
seq_time = time.perf_counter() - start
print(f"  Total: {seq_time:.2f}s\n")

# Concurrent execution
print("=== Concurrent (asyncio.gather) ===")
start = time.perf_counter()
con_results = await main_concurrent()
con_time = time.perf_counter() - start
print(f"  Total: {con_time:.2f}s")
print(f"  Speedup: {seq_time / con_time:.1f}x")

## Async Patterns: Tasks and Timeouts

`asyncio.create_task()` schedules a coroutine to run in the background.
`asyncio.wait_for()` adds a timeout to any awaitable. These patterns
give fine-grained control over concurrent async operations.

In [ ]:
import asyncio


async def slow_operation(name: str, delay: float) -> str:
    """A coroutine that might take too long."""
    await asyncio.sleep(delay)
    return f"{name} completed"


async def demonstrate_tasks() -> None:
    """Show create_task for background scheduling."""
    print("=== asyncio.create_task ===")

    # Tasks start running immediately when created
    task_a = asyncio.create_task(slow_operation("Task-A", 0.3))
    task_b = asyncio.create_task(slow_operation("Task-B", 0.1))

    print(f"  Task-A done? {task_a.done()}")
    print(f"  Task-B done? {task_b.done()}")

    # Await results
    result_a = await task_a
    result_b = await task_b
    print(f"  Results: {result_a}, {result_b}")


async def demonstrate_timeout() -> None:
    """Show wait_for with timeout handling."""
    print("\n=== asyncio.wait_for with timeout ===")

    # This completes in time
    try:
        result = await asyncio.wait_for(slow_operation("Fast", 0.1), timeout=1.0)
        print(f"  Success: {result}")
    except asyncio.TimeoutError:
        print("  Timed out!")

    # This exceeds the timeout
    try:
        result = await asyncio.wait_for(slow_operation("Slow", 5.0), timeout=0.2)
        print(f"  Success: {result}")
    except asyncio.TimeoutError:
        print("  Timed out! (as expected for the slow operation)")


async def demonstrate_exception_handling() -> None:
    """Show exception handling with gather."""
    print("\n=== Exception handling in gather ===")

    async def might_fail(name: str, should_fail: bool) -> str:
        await asyncio.sleep(0.1)
        if should_fail:
            raise ValueError(f"{name} failed!")
        return f"{name} succeeded"

    # return_exceptions=True prevents one failure from cancelling others
    results = await asyncio.gather(
        might_fail("A", False),
        might_fail("B", True),
        might_fail("C", False),
        return_exceptions=True,
    )

    for i, result in enumerate(results):
        if isinstance(result, Exception):
            print(f"  Task {i}: FAILED - {result}")
        else:
            print(f"  Task {i}: {result}")


await demonstrate_tasks()
await demonstrate_timeout()
await demonstrate_exception_handling()

## When to Use Threads vs Processes vs Asyncio

| Criterion | `threading` | `multiprocessing` | `asyncio` |
|---|---|---|---|
| **Best for** | I/O-bound (moderate concurrency) | CPU-bound (true parallelism) | I/O-bound (high concurrency) |
| **GIL impact** | Limited by GIL for CPU work | Bypasses GIL (separate processes) | Single-threaded, no GIL issue |
| **Memory** | Shared (needs locks) | Separate (needs IPC) | Shared (no locks needed) |
| **Overhead** | Low (lightweight) | High (process creation) | Very low (coroutines) |
| **Scalability** | ~100s of threads | ~10s of processes | ~10,000s of coroutines |
| **Debugging** | Hard (race conditions) | Moderate (isolation helps) | Easier (single thread) |
| **Use case** | File I/O, simple web scraping | Data processing, math | Web servers, API clients |

In [ ]:
# Summary: choosing the right concurrency model
from dataclasses import dataclass
from enum import Enum, auto


class WorkloadType(Enum):
    IO_BOUND = auto()
    CPU_BOUND = auto()


class ConcurrencyLevel(Enum):
    LOW = auto()       # < 100 concurrent tasks
    HIGH = auto()      # 100s to 1000s of concurrent tasks


@dataclass(frozen=True)
class ConcurrencyAdvice:
    model: str
    module: str
    reasoning: str


def recommend_concurrency(
    workload: WorkloadType,
    concurrency: ConcurrencyLevel,
) -> ConcurrencyAdvice:
    """Recommend the appropriate concurrency model."""
    if workload == WorkloadType.CPU_BOUND:
        return ConcurrencyAdvice(
            model="Multiprocessing",
            module="concurrent.futures.ProcessPoolExecutor",
            reasoning="CPU-bound work needs separate processes to bypass the GIL",
        )
    # I/O-bound
    if concurrency == ConcurrencyLevel.HIGH:
        return ConcurrencyAdvice(
            model="Asyncio",
            module="asyncio",
            reasoning="High-concurrency I/O benefits from lightweight coroutines",
        )
    return ConcurrencyAdvice(
        model="Threading",
        module="concurrent.futures.ThreadPoolExecutor",
        reasoning="Moderate I/O concurrency is well-served by threads",
    )


# Example scenarios
scenarios: list[tuple[str, WorkloadType, ConcurrencyLevel]] = [
    ("Image processing pipeline", WorkloadType.CPU_BOUND, ConcurrencyLevel.LOW),
    ("Web scraper (50 URLs)", WorkloadType.IO_BOUND, ConcurrencyLevel.LOW),
    ("Async web server", WorkloadType.IO_BOUND, ConcurrencyLevel.HIGH),
    ("ML model training", WorkloadType.CPU_BOUND, ConcurrencyLevel.HIGH),
]

for name, workload, level in scenarios:
    advice = recommend_concurrency(workload, level)
    print(f"{name}:")
    print(f"  Model:  {advice.model} ({advice.module})")
    print(f"  Why:    {advice.reasoning}\n")